# **AI Agents from Scratch**
*   **REQUIRED:** In Colab, connect to at minimum T4 GPU

* **ARTICLE:** https://medium.com/@ajarkas/if-you-know-python-ai-agents-are-simple-from-scratch-3199eea5e43c






In [1]:
#@title Install Dependencies
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

import math, gc, torch, warnings
from unsloth import FastLanguageModel
from transformers import TextStreamer
import re
import pandas as pd
import numpy as np
warnings.filterwarnings("ignore")

In [ ]:
#@title Loading the LLM

#Recommened Model: Qwen3-14B (4bit) ~11GB VRAM
model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/Qwen3-14B",   #Qwen3-8B also available (smaller model)
    max_seq_length = 4096, #adjust depending on RAM & Task
    load_in_4bit   = True,
    load_in_8bit   = False,
)

model.eval()


In [3]:
#@title Function to Generate LLM Response
def generate_response(message, messages):
  """
  Inputs: message <string> -> latest user message,
          messages <List> -> chat history

  Output: LLM response <string>

  """
  messages.append({"role": "user", "content": message})
  prompt = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True,
      enable_thinking=False,
  )

  inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

  with torch.inference_mode():
      output_ids = model.generate(
          **inputs,
          max_new_tokens=5120, #adjust depending on RAM & Task
          temperature=0.7,
          top_p=0.8,
          top_k=20,
      )

  gen_ids = output_ids[0, inputs["input_ids"].shape[-1]:]
  result_text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

  if "END_TASK" in result_text:
    print(str(result_text).replace("END_TASK", "").strip())
  elif '<tool>' in result_text:
    print(f"""Accessing Tool: {re.findall(r'<tool>(.*?)</tool>', result_text)[0]}""")
    print('\n')
  else: None

  messages.append({"role": "assistant", "content": result_text})
  return result_text

In [4]:
#@title Defining your custom tools
!pip install -q yfinance #Yahoo Finance (Stock API)

def get_current_date():
  """
  Get the current date in the format 'YYYY-MM-DD'.

  parameters: None
  returns: string in the format 'YYYY-MM-DD'
  """
  import datetime
  return datetime.datetime.now().strftime("%Y-%m-%d")

def get_stock_data(ticker_symbol: str, start_date: str, end_date: str):
  """
  Get historical stock data for a given ticker symbol. Assuming all dates are valid. if not valid, will return None

  Parameters:
  - ticker_symbol (str): The ticker symbol of the stock.
  - start_date (str): The start date in the format 'YYYY-MM-DD'.
  - end_date (str): The end date in the format 'YYYY-MM-DD'.

  Returns:
  - string form of pandas.DataFrame
  """
  import yfinance as yf
  from datetime import datetime, timedelta
  warnings.filterwarnings("ignore", module=r"yfinance\..*")

  #pad time window as some dates are invalid trading days
  start_date = datetime.strptime(start_date, '%Y-%m-%d') - timedelta(days=4)
  end_date = datetime.strptime(end_date, '%Y-%m-%d') + timedelta(days=4)

  with warnings.catch_warnings():
      warnings.filterwarnings("ignore", category=FutureWarning, module="yfinance")
      data = yf.download(ticker_symbol, start=start_date, end=end_date,
                        progress=False, auto_adjust=True)
  if data.empty:
    return "Date used is not trading day, try ± 4 day around the start & end date"
  return str(data.Close.to_dict())

def get_current_time():
    """
    Returns current time in HH:MM:SS string format
    """
    import datetime
    return datetime.datetime.now().strftime("%H:%M:%S")


def save_csv(data, directory="", filename="output.csv"):
  """
  Save a pandas DataFrame to a CSV file in a specified directory.

  Parameters:
  data <dict>,
  directory <string>,
  filename <string>

  Output: Success or Failure <string>

  """
  import pandas as pd
  import os

  if directory != "" and not os.path.exists(directory):
    os.makedirs(directory)

  try:
    if isinstance(data, dict):
      data = pd.DataFrame(data)
    data.to_csv(os.path.join(directory, filename), index=False)
    return "Success"
  except:
    return "Failure"

In [8]:
#@title Assigning Toolset (To call on tools for later)
tools = {
          'get_current_date': get_current_date,
          'get_stock_data': get_stock_data,
          'save_csv': save_csv,
          #'get_current_time': get_current_time,
         }


#Context of your tools' functionality, parameters, and outputs for LLM usage
tool_docs = "\n".join([f"* {i.__name__}: {i.__doc__}" for i in tools.values()])
print(tool_docs)

* get_current_date: 
  Get the current date in the format 'YYYY-MM-DD'.

  parameters: None
  returns: string in the format 'YYYY-MM-DD'
  
* get_stock_data: 
  Get historical stock data for a given ticker symbol. Assuming all dates are valid. if not valid, will return None

  Parameters:
  - ticker_symbol (str): The ticker symbol of the stock.
  - start_date (str): The start date in the format 'YYYY-MM-DD'.
  - end_date (str): The end date in the format 'YYYY-MM-DD'.

  Returns:
  - string form of pandas.DataFrame
  
* save_csv: 
  Save a pandas DataFrame to a CSV file in a specified directory.

  Parameters:
  data <dict>,
  directory <string>,
  filename <string>

  Output: Success or Failure <string>

  


In [16]:
#@title System Prompt (Default prompt for LLM aka Instructions)

system_prompt = f"""You're an AI assistant with access to these tools (python functions):\n
{tool_docs}

** Given a prompt from the user, assess calling those tools if needed and in which synchronous order.
** If no tool needed (such as for basic calculations), can do those calculations on your own.
** If tools are needed, then call one tool at a time in order of operations.
** Only call one tool per response.
** You will receive a response from the tool after that.
** After you receive that tool's response, call the next necessary tool until you have completed and answered the user's question.
** In your final response, end it with "END_TASK".

ALWAYS ANSWER THE USER DIRECTLY, CONCISELY, and FORTHRIGHTLY.

How to call tools:

- <tool>get_current_time</tool>
- <tool>get_stock_data</tool> <args>{{'ticker_symbol': 'AAPL', 'start_date': '2024-01-01', 'end_date': '2024-01-31'}}</args>

"""


print(system_prompt)

You're an AI assistant with access to these tools (python functions):

* get_current_date: 
  Get the current date in the format 'YYYY-MM-DD'.

  parameters: None
  returns: string in the format 'YYYY-MM-DD'
  
* get_stock_data: 
  Get historical stock data for a given ticker symbol. Assuming all dates are valid. if not valid, will return None

  Parameters:
  - ticker_symbol (str): The ticker symbol of the stock.
  - start_date (str): The start date in the format 'YYYY-MM-DD'.
  - end_date (str): The end date in the format 'YYYY-MM-DD'.

  Returns:
  - string form of pandas.DataFrame
  
* save_csv: 
  Save a pandas DataFrame to a CSV file in a specified directory.

  Parameters:
  data <dict>,
  directory <string>,
  filename <string>

  Output: Success or Failure <string>

  

** Given a prompt from the user, assess calling those tools if needed and in which synchronous order. 
** If no tool needed (such as for basic calculations), can do those calculations on your own.
** If tools a

In [17]:
#@title Calling relevant Tool given LLM output string

import re
def call_tool(tool_blurb):
  """
  Input: LLM output (string) with tool call (i.e §tool_name§)
  Operation: Transforms the LLM output string to a tool call operation in python
  Output: Result Tool call operation (i.e get_current_time -> 12:59:00 PM)

  """

  #Parse/Extract tool name and tool parameters
  tool_name = re.findall(r'<tool>(.*?)</tool>', tool_blurb)[0] if '<tool>' in tool_blurb else None
  tool_args = re.findall(r'<args>(.*?)</args>', tool_blurb)[0] if '<args>' in tool_blurb else None

  if tool_name and tool_args:
    return tools[tool_name](**eval(tool_args))
  elif tool_name:
    return tools[tool_name]()
  else:
    return "No tool call found"


llm_output = "<tool>get_current_date</tool>" #sample output for demonstration purposes
call_tool(llm_output)

'2025-08-17'

In [19]:
#@title LLM Loop (User Prompt -> LLM Thinking, Tool Calling, ... -> Final Answer)

messages = [{"role": "system", "content": system_prompt}]

message = """I purchased 234 stock shares of NVIDIA on 4/7/2025, I sold all shares 5 days ago.

#1 Calculate how much money did I make or lose?

#2 Save that buy/sell information as orders.csv in current directory
with columns date, ticker, price, quantity, action (buy/sell), total_value (price * quantity)
"""

#initializers
resp = ""
count = 0

#Loops, calls tools when needed, stops when final answer is produced noted by tag 'END_TASK'
while 'END_TASK' not in resp:
  resp = generate_response(message, messages)

  if '<tool>' in resp:
    message = f"Tool output: {call_tool(resp)}"
    count += 1
  else:
    message = ""

  #If agent falls in endless loop, cease
  count += 1
  if count > 20:
    "Timed Out"
    break


Accessing Tool: get_current_date


Accessing Tool: get_stock_data


Accessing Tool: get_stock_data


Accessing Tool: save_csv


The buy/sell information has been successfully saved as "orders.csv" in the current directory.

For the calculation of profit or loss:

- The purchase price per share was approximately $97.63 on 2025-04-07.
- The selling price per share was approximately $183.16 on 2025-08-12.

Profit per share = Selling price - Purchase price = $183.16 - $97.63 = $85.53.
Total profit = Profit per share * Quantity = $85.53 * 234 = $20,012.02.

You made a profit of $20,012.02.


In [20]:
 #@title Output from AI Agent
import pandas as pd
pd.read_csv('orders.csv')

,date,ticker,price,quantity,action,total_value
0,2025-04-07,NVDA,97.633217,234,buy,22846.172745
1,2025-08-12,NVDA,183.160004,234,sell,42859.440857


In [30]:
#@title Full Chat History
[print(f"{m['role'].upper()}: {m['content']}") for m in messages]

SYSTEM: You're an AI assistant with access to these tools (python functions):

* get_current_date: 
  Get the current date in the format 'YYYY-MM-DD'.

  parameters: None
  returns: string in the format 'YYYY-MM-DD'
  
* get_stock_data: 
  Get historical stock data for a given ticker symbol. Assuming all dates are valid. if not valid, will return None

  Parameters:
  - ticker_symbol (str): The ticker symbol of the stock.
  - start_date (str): The start date in the format 'YYYY-MM-DD'.
  - end_date (str): The end date in the format 'YYYY-MM-DD'.

  Returns:
  - string form of pandas.DataFrame
  
* save_csv: 
  Save a pandas DataFrame to a CSV file in a specified directory.

  Parameters:
  data <dict>,
  directory <string>,
  filename <string>

  Output: Success or Failure <string>

  

** Given a prompt from the user, assess calling those tools if needed and in which synchronous order. 
** If no tool needed (such as for basic calculations), can do those calculations on your own.
** If

[None, None, None, None, None, None, None, None, None, None, None]